In [2]:
from eventregistry import *
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions
import json
import pandas as pd
from newspaper import Article

In [23]:
global_df = pd.DataFrame()

eventregistry_api = 'aa1174a5-a633-492d-a182-28d59fc28a34'
watson_api = 'VDaX2sFbNmoKB3gQit67DgC_2y5hnsEwLA1RU49Ko65H'

def getInputArticleKeywords(user_url):
#     url = user_url.decode('utf-8')
    article = Article(user_url)
    article.download()
    article.parse()
    article.nlp()

    keywords = article.keywords
    kws = []
    for word in keywords:
        if len(kws) < 10:
            kws.append(word)

    return kws


def getArticles(keywords):
    global global_df
    er = EventRegistry(apiKey = eventregistry_api)
    q = QueryArticlesIter(
        keywords = QueryItems.OR(keywords))
        # keywordsLoc = "title",
        # ignoreKeywords = "SpaceX",
        # sourceUri = "nytimes.com")
    q.setRequestedResult(RequestArticlesInfo(sortBy="rel"))
    res = q.execQuery(er, sortBy = "rel", maxItems = 500)
    local_df = pd.DataFrame()
    index = 0
    for article in res:
        data = {
            'source': article['source']['title'].encode('utf-8'),
            'url' : article['url'].encode('utf-8'),
            'text' : article['body'].encode('utf-8')
        }
        local_df = pd.concat([local_df, pd.DataFrame(data,index=[index])])
        index += 1
    
    # append to global dataframe
    global_df = pd.concat([global_df,local_df])
    
def getKeywords(url):
    natural_language_understanding = NaturalLanguageUnderstandingV1(
        version='2018-11-16',
        iam_apikey=watson_api,
        url='https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
    )

    response = natural_language_understanding.analyze(
        url=url,
        features=Features(keywords=KeywordsOptions())).get_result()
#            features=Features(entities=EntitiesOptions(),
#                       keywords=KeywordsOptions())).get_result()
    keywords = []
    
    for keyword in response['keywords']:
        if keyword['relevance'] > 0.8 and len(keywords) < 10:
            keywords.append(keyword['text'].encode('utf-8'))
    return keywords
    
# given the url of 
def webscrapeMain(url):
    global global_df
    # get keywords 
    kws = getInputArticleKeywords(url)
#     kws = getKeywords(url)
    print("The keywords are: {}".format(kws))

    getArticles(kws)
    print(global_df)
    global_df = global_df.reset_index(drop=True)
    global_df.to_csv('articles.csv')
    
# run
url = 'https://www.theguardian.com/world/2018/may/01/the-us-and-afghanistan-cant-win-the-war-cant-stop-it-cant-leave'
webscrapeMain(url)
print("Finished scraping for articles related to the article")


The keywords are: ['forces', 'cant', 'war', 'afghanistan', 'trumps', 'trump', 'peace', 'taliban', 'killed', 'leave']
using user provided API key for making requests
Event Registry host: http://eventregistry.org
Text analytics host: http://analytics.eventregistry.org
                                     source  \
0                   b'southasiajournal.net'   
1                   b'southasiajournal.net'   
2                     b'The Nigerian Voice'   
3             b'International Crisis Group'   
4                         b'Foreign Policy'   
5                   b'southasiajournal.net'   
6                   b'southasiajournal.net'   
7                          b'The Intercept'   
8                     b'The New York Times'   
9                                 b'RTE.ie'   
10                          b'StrategyPage'   
11                               b'alaraby'   
12                          b'StrategyPage'   
13                  b'Global Village Space'   
14                          

In [4]:
import nltk
import requests
from IPython.display import display, HTML


def azureSearch(claim):
    key = '34d4fdab594e46c2b8f4b497042a7260'
    search_url = "https://api.cognitive.microsoft.com/bing/v7.0/search"

    headers = {"Ocp-Apim-Subscription-Key" : key}
    params  = {"q": claim, "textDecorations":True, "textFormat":"HTML"}
    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()
    rows = "\n".join(["""<tr>
                       <td><a href=\"{0}\">{1}</a></td>
                       <td>{2}</td>
                     </tr>""".format(v["url"],v["name"],v["snippet"]) \
                  for v in search_results["webPages"]["value"]])
    display(HTML("<table>{0}</table>".format(rows)))
#     print(search_results)
#     search_service = PyMsCognitiveWebSearch(key, claim)
#     first_three_result = search_service.search(limit=3, format='json') #1-50

#     urls = []
#    # To get individual result json:
#     for i in first_three_result:
#         urls.append(i.url.encode('utf-8'))
#     return urls

azureSearch('राम मंदिर नहीं बनेगा ')

राम मंदिर अयोध्या में नहीं तो और कहां बनेगा - YouTube,राम मंदिर अयोध्या में नहीं तो और कहां बनेगा ... आपने ऐसा कभी नहीं ... राम मंदिर Vs ...
"राम मंदिर नहीं, अयोध्या में पहले बनेगा राम का स्टेचू ...","राम मंदिर नहीं, अयोध्या में पहले बनेगा राम का स्टेचू ! HW News Network Loading..."
बाबा रामदेव बोले- अब नहीं बना तो कभी नहीं बनेगा राम मंदिर ...,योग गुरु स्वामी रामदेव बाबा ने अमर उजाला संवाद में तमाम सवालों के जवाब दिए। उन्होंने राम मंदिर मुद्दे पर कहा कि अगर अब नहीं बना तो मंदिर कभी नहीं बनेगा।
"राम मंदिर बनेगा, नहीं बनेगा, कब बनेगा? इन सवालों से पहले ...",भगवान राम से जुड़ी कुछ ऐसी बातें जो शायद राम भक्त भी न बता पाएं।जिसके द्वारा आप राम को अच्छे से जान सकते हैं। ये तथ्य आज से पहले आपको किसी ने नहीं बताया ...
"राम मंदिर का पता नहीं, लेकिन मंदिर जैसा बनेगा अयोध्या का ...","राम मंदिर का पता नहीं, लेकिन मंदिर जैसा बनेगा अयोध्या का रेलवे स्टेशन By Kanwhizz Times"
नही बनेगा राम मंदिर फैसले की सुनवाई में आ रही है कठिनाइयां ...,नही बनेगा राम मंदिर फैसले की सुनवाई में आ रही है कठिनाइयां। ... हैं हमें बीजेपी राम मंदिर को लेकर दिखाओ कि नहीं लगती है और चुनाव जाने के ...
नहीं बनेगा राम मंदिर बनेगा क्षत्रिय राम का राज महल -राजा ...,"नहीं बनेगा राम मंदिर बनेगा क्षत्रिय राम का राज महल -राजा राजेंद्र सिंह(अखिल भारतीय क्षत्रिय महासभा) Posted by: Shakti Singh Chandel October 11, 2018 620 Views"
राम मंदिर बनेगा या नहीं: मुंबई से आई शिव सैनिकों ने मोदी ...,2019 के लोकसभा चुनाव से पहले राम मंदिर का मुद्दा लगातार गर्म होता जा रहा है ऐसे में सबसे बड़ा सवाल यही है कि राम मंदिर बनेगा या फिर नहीं?
"राम मंदिर नहीं, अयोध्या में पहले बनेगा राम का स्टेचू ...","राम मंदिर नहीं, अयोध्या में पहले बनेगा राम का स्टेचू ! ... अयोध्या में पहले बनेगा राम का स्टेचू ! Published. 1 month ago. on. November 25, 2018. By."
भारत में राम मंदिर नहीं बनेगा तो क्या पाकिस्तान में बनेगा ...,बेंगलुरू. कर्नाटक में कांग्रेस के विधायक और पूर्व मंत्री रोशन बेग ने गुरूवार को सवाल किया कि राम मंदिर भारत में नहीं बनेगा तो क्या पाकिस्तान में बनेगा?


In [ ]:
df1 = pd.read_csv('100_articles.csv')
df1.columns = ['id','source','text',"""'title""" 'url']

bodies = df1.loc[:,['id','text']]
bodies.to_csv('bodies.csv')

claim = [claim] * len(global_df.index)
claims = pd.DataFrame(claim)
claims['id'] = range(len(global_df.index))
claims.to_csv('claims.csv')

urls = df1.loc[:,['id','source','url']]
urls.to_csv('url.csv')